# Import Packages

In [28]:
import pandas as pd
import numpy as np
import datetime as dt
import requests
import jsonb

print('libraries read!')

libraries read!


# Test

In [4]:
api_key = "717ab7b42cfda160f09acd77a784e0ca"
lat = str("49.504587")
lon = str("5.944140")
time_stamp = str("2019-03-21 15:00:00")
time_stamp = pd.to_datetime(time_stamp,infer_datetime_format=True).timestamp()
time_stamp = int(time_stamp)
querystring = {"lang":"en","units":"si"}

In [5]:
url = "https://api.darksky.net/forecast/%s/%s,%s,%s?exclude=minutely,hourly,daily,alerts,flags" % (api_key, lat, lon, time_stamp)

In [6]:
response = requests.request("GET", url, params=querystring)
data = json.loads(response.text)
data


{'latitude': 49.504587,
 'longitude': 5.94414,
 'timezone': 'Europe/Luxembourg',
 'currently': {'time': 1553180400,
  'temperature': 15.45,
  'apparentTemperature': 15.45,
  'dewPoint': -2.99,
  'humidity': 0.28,
  'windSpeed': 4.6,
  'windBearing': 60,
  'visibility': 10.003},
 'offset': 1}

In [7]:
data = data['currently']
data

{'time': 1553180400,
 'temperature': 15.45,
 'apparentTemperature': 15.45,
 'dewPoint': -2.99,
 'humidity': 0.28,
 'windSpeed': 4.6,
 'windBearing': 60,
 'visibility': 10.003}

In [8]:
df_weather = pd.DataFrame.from_dict(data.items()).set_index(0)
df_weather = df_weather.transpose()
df_weather

,time,temperature,apparentTemperature,dewPoint,humidity,windSpeed,windBearing,visibility
1,1.553180e+09,15.45,15.45,-2.99,0.28,4.6,60.0,10.003


# File test

In [15]:
base_dir = "./"

In [16]:
df = pd.read_csv(base_dir + 'data/test.csv').iloc[:,1:]
#df['Planned_Departure'] = pd.to_datetime(df['Planned_Departure'],infer_datetime_format=True)
df

,Origin_Latitude,Origin_Longitude,Planned_Departure
0,49.493889,6.108889,2019-03-07 15:00:00
1,49.493889,6.108889,2019-03-14 15:00:00
2,49.493889,6.108889,2019-03-21 15:00:00
3,49.493889,6.108889,2019-03-28 15:00:00
4,51.340170,3.221882,2019-03-04 05:25:00
5,51.340170,3.221882,2019-03-11 05:25:00
6,51.340170,3.221882,2019-03-18 05:25:00
7,51.340170,3.221882,2019-03-25 05:25:00
8,49.493889,6.108889,2019-03-04 13:59:00
9,49.493889,6.108889,2019-03-11 13:59:00


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Origin_Latitude    10 non-null     float64
 1   Origin_Longitude   10 non-null     float64
 2   Planned_Departure  10 non-null     object 
dtypes: float64(2), object(1)
memory usage: 368.0+ bytes


In [25]:
from weather_request import get_weather_data
key = "717ab7b42cfda160f09acd77a784e0ca"

In [26]:
df[['temperature','apparentTemperature','dewPoint','humidity','windSpeed','windBearing','cloudCover','uvIndex','visibility']] = \
df.apply(lambda x: get_weather_data(x['Origin_Latitude'],x['Origin_Longitude'],x['Planned_Departure'],key), axis=1, result_type='expand')  # result_type='expand' is for unpacking all values in all columns, and not assign all values in a list

In [27]:
df.head()

,Origin_Latitude,Origin_Longitude,Planned_Departure,temperature,apparentTemperature,dewPoint,humidity,windSpeed,windBearing,cloudCover,uvIndex,visibility
0,49.493889,6.108889,2019-03-07 15:00:00,7.56,2.91,5.00,0.84,10.30,240.0,0.75,1.0,10.003
1,49.493889,6.108889,2019-03-14 15:00:00,7.56,3.03,6.00,0.90,9.80,251.0,0.75,1.0,5.002
2,49.493889,6.108889,2019-03-21 15:00:00,15.56,15.56,-2.99,0.28,4.60,60.0,NaN,NaN,10.003
3,49.493889,6.108889,2019-03-28 15:00:00,14.56,14.56,2.00,0.43,4.11,50.0,0.19,2.0,10.003
4,51.340170,3.221882,2019-03-04 05:25:00,7.75,2.60,4.63,0.81,13.03,235.0,0.73,0.0,9.904
